## LATENT DIRICHLET ALLOCATION

https://medium.com/@lettier/how-does-lda-work-ill-explain-using-emoji-108abf40fa7d
    
https://www.youtube.com/watch?v=DWJYZq_fQ2A

In [3]:
import random
import matplotlib.pyplot as plt
from collections import Counter
import numpy as np

topic1 = {
    'id': 'sports', 
    'words': ['football', 'basketball', 'gol', 'play', 'match', 'space'],
    'weights': [4, 1, 1, 1, 1, 2]
}
topic2 = {
    'id': 'politics', 
    'words': ['president', 'interview', 'twitter', 'television', 'debate', 'space'],
    'weights': [2, 1, 4, 1, 1, 1]
}
topic3 = {
    'id': 'science', 
    'words': ['science', 'molecula', 'debate', 'space'],
    'weights': [1, 1, 1, 1]
}

topics = [topic1, topic2, topic3]


# topic1 = {
#     'id': 'sports', 
#     'words': ['football', 'basketball', 'match'],
#     'weights': [1,1,1]
# }
# topic2 = {
#     'id': 'politics', 
#     'words': ['president', 'interview', 'twitter'],
#     'weights': [1, 1, 1]
# }
# topic3 = {
#     'id': 'science', 
#     'words': ['science', 'molecula', 'space'],
#     'weights': [1, 1, 1]
# }

topics = [topic1, topic2, topic3]


# topic1 = {
#     'id': 'dogs', 
#     'words': ['guau', 'guau2'],
#     'weights': [1, 1]
# }
# topic2 = {
#     'id': 'cats', 
#     'words': ['miaw', 'miaw2'],
#     'weights': [1, 1]
# }

# topics = [topic1, topic2]


## Generar documentos 


In [12]:
def generate_documents(topics, num_docs=100, topics_per_doc=2, words_per_doc=1000):
    documents = []
    for i in range(num_docs):
        # first select several topics
        doc_topics = random.sample(topics, topics_per_doc)

        # asign a weight to each topic of the document
        r = [random.random() for _ in range(0, topics_per_doc)]
        s = sum(r)
        doc_topics_weights = [ j/s for j in r ]
        
        # now sample the topics to generate the document. 
        # NOTE: this is an inefficient version for teaching purposes
        doc_words = [] 
        for _ in range(words_per_doc):
            topic = random.choices(doc_topics, doc_topics_weights)[0]  # select one topic
            doc_words = doc_words + random.choices(topic['words'], topic['weights'], k=1)  # choose one word
        
        documents.append({'id': i, 'words': doc_words, 'topics_weights': [(t[0]['id'], t[1]) for t in zip(doc_topics, doc_topics_weights)]})
        
        words_idx = list(set([word for doc in documents for word in doc['words']]))
        docs_idx = [doc['id'] for doc in documents]
        topics_idx = [t['id'] for t in topics]
    return documents, docs_idx, words_idx, topics_idx


docs, docs_idx, words_idx, topics_idx = generate_documents(topics, num_docs=100, topics_per_doc=2, words_per_doc=100)
docs

[{'id': 0,
  'words': ['twitter',
   'twitter',
   'interview',
   'twitter',
   'football',
   'debate',
   'television',
   'twitter',
   'play',
   'football',
   'television',
   'space',
   'space',
   'twitter',
   'twitter',
   'television',
   'space',
   'space',
   'match',
   'twitter',
   'football',
   'twitter',
   'space',
   'television',
   'president',
   'president',
   'gol',
   'space',
   'twitter',
   'president',
   'television',
   'twitter',
   'twitter',
   'football',
   'twitter',
   'space',
   'football',
   'president',
   'television',
   'space',
   'twitter',
   'twitter',
   'twitter',
   'debate',
   'football',
   'football',
   'interview',
   'president',
   'interview',
   'football',
   'debate',
   'president',
   'president',
   'play',
   'space',
   'match',
   'space',
   'twitter',
   'twitter',
   'television',
   'match',
   'space',
   'debate',
   'debate',
   'twitter',
   'space',
   'president',
   'match',
   'space',
   'presiden

## Generar tablas auxiliares: words vs topics, docs vs topics

In [18]:
def words_vs_topics_matrix(docs, words_idx, ntopics):
    m = np.zeros((len(words_idx), ntopics))
    for doc in docs:
        for a in doc['assignment']:
            word_j = words_idx.index(a['word'])
            m[word_j, a['topic']] += 1
    return m


def docs_vs_topics_matrix(docs, ntopics):
    m = np.zeros((len(docs), ntopics))
    for doc in docs:
        doc_i = docs.index(doc)
        for a in doc['assignment']:
            m[doc_i, a['topic']] += 1
    return m


def random_assignment(docs, ntopics):
    for doc in docs:
        assignment = zip(doc['words'], np.random.randint(0, ntopics, len(doc['words'])))  # get random initial assignment for each word in each document
        doc['assignment'] = [{'word': t[0], 'topic': t[1]} for t in assignment]
    

test_docs = [
    {'id': 0, 'assignment': [{'word': 'A', 'topic': 0}, {'word': 'AA', 'topic': 0}]},
    {'id': 1, 'assignment': [{'word': 'B', 'topic': 1}, {'word': 'BB', 'topic': 1}]},
]

# TEST
words_vs_topics_matrix(test_docs, words_idx=['A', 'AA', 'B', 'BB'], ntopics=2) == np.array([[1,0],[1,0],[0,1],[0,1]])


array([[ True,  True],
       [ True,  True],
       [ True,  True],
       [ True,  True]])

## Dirichlet allocation versión lenta

In [19]:
def prob_w_belongs_t_in_d(words_vs_topics, docs_vs_topics, d, w, t, old_topic, alpha, beta, debug=False):
    # P(w belongs topic) = P(word w | topic t) * P(topic t | doc d)
    # P(w belongs topic) = Proporcion(word w en el topic t) * Proporcion(topic t en el doc d)
    # prob_w_belongs_t = words_vs_topics[w, t]/words_vs_topics[:, t] * docs_vs_topics[d, t]/sum(docs_vs_topic[d, :])
    nwords, ntopics = words_vs_topics.shape
    ndocs, _ = docs_vs_topics.shape
        
    if debug:
        print("words_vs_topics=\n", words_vs_topics, words_vs_topics[w, t], (w,t))
        print("docs_vs_topics=\n", docs_vs_topics, docs_vs_topics[d, t], (d,t))
        print("t=", t)
        print("d=", d)
        print("w=", w)
        print("old_topic=", old_topic)
    
    if old_topic == t:
        # Note: as we are calculating the new topic assignation for w, we need to eliminate the
        # old assignation. That is why we rest -1 in those calculations
        assert words_vs_topics[w, t] > 0
        assert docs_vs_topics[d, t] > 0
        if debug:
            print("REMOVING OLD ASIGNATION")
            print((words_vs_topics[w, t]-1 + beta)/(sum(words_vs_topics[:, t])-1 + beta*nwords))
            print((docs_vs_topics[d, t]-1 + alpha)/(sum(docs_vs_topics[d, :])-1 + alpha*ntopics))
            input()
        return (words_vs_topics[w, t]-1 + beta)/(sum(words_vs_topics[:, t])-1 + beta*nwords) \
               * (docs_vs_topics[d, t]-1 + alpha)/(sum(docs_vs_topics[d, :])-1 + alpha*ntopics)
    else:
        if debug:
            print((words_vs_topics[w, t] + beta)/(sum(words_vs_topics[:, t]) + beta))
            print((docs_vs_topics[d, t] + alpha)/(sum(docs_vs_topics[d, :]) + alpha*ntopics))
            input()
        return (words_vs_topics[w, t] + beta)/(sum(words_vs_topics[:, t]) + beta) \
               * (docs_vs_topics[d, t] + alpha)/(sum(docs_vs_topics[d, :]) + alpha*ntopics)


def dirichlet_allocation(docs, docs_idx, words_idx, ntopics, niter=400, alpha=0.5, beta=0.01, debug=False):
    ndocs, nwords = len(docs_idx), len(words_idx)

    random_assignment(docs, ntopics)

    for _ in range(niter):

        words_vs_topics = words_vs_topics_matrix(docs, words_idx, ntopics)
        # print(words_vs_topics)

        docs_vs_topics = docs_vs_topics_matrix(docs, ntopics)
        # print(docs_vs_topics)

        for doc in docs:
            doc_i = docs_idx.index(doc['id'])
            for assign in doc['assignment']:
                word = assign['word']
                word_j = words_idx.index(word)
                old_topic = assign['topic']
                prob_by_topic = [0] * ntopics  # unnormalized
                for topic_k in range(ntopics):
                    if debug:
                        print(f"==>> doc {doc_i}, word {word_j}, calculating for topic {topic_k} (old topic was {old_topic})")
                    p = prob_w_belongs_t_in_d(words_vs_topics, docs_vs_topics, doc_i, word_j, topic_k, old_topic=old_topic, alpha=alpha, beta=beta, debug=debug)
                    prob_by_topic[topic_k] = p

                new_topic = random.choices(range(ntopics), prob_by_topic, k=1)[0]
                assign['topic'] = new_topic

    return docs


docs = dirichlet_allocation(docs, docs_idx, words_idx, ntopics=3, niter=100)
# docs

In [6]:
def normalize_counter(c):
    total = sum(c.values(), 0.0)
    for key in c:
        c[key] /= total
    return c

for (i, d) in enumerate(docs):
    print(i, d['topics_weights'])
    topic_list = Counter([a['topic'] for a in d['assignment']])
    a = normalize_counter(Counter(topic_list))
    print(i, sorted(a.items()))

0 [('science', 0.24456755248379008), ('sports', 0.75543244751621)]
0 [(0, 0.01), (1, 0.77), (2, 0.22)]
1 [('sports', 0.35727722282000307), ('politics', 0.6427227771799969)]
1 [(0, 0.62), (1, 0.37), (2, 0.01)]
2 [('sports', 0.542989916612604), ('science', 0.45701008338739607)]
2 [(1, 0.56), (2, 0.44)]
3 [('science', 0.39538758109917665), ('sports', 0.6046124189008233)]
3 [(1, 0.67), (2, 0.33)]
4 [('science', 0.07236994958194534), ('sports', 0.9276300504180547)]
4 [(0, 0.01), (1, 0.92), (2, 0.07)]
5 [('sports', 0.43887033509303514), ('politics', 0.5611296649069649)]
5 [(0, 0.58), (1, 0.42)]
6 [('sports', 0.11407328212216619), ('politics', 0.8859267178778338)]
6 [(0, 0.92), (1, 0.08)]
7 [('science', 0.6406622574014711), ('sports', 0.3593377425985289)]
7 [(1, 0.35), (2, 0.65)]
8 [('science', 0.09343523903964185), ('sports', 0.9065647609603581)]
8 [(1, 0.93), (2, 0.07)]
9 [('sports', 0.4143566061847329), ('science', 0.5856433938152672)]
9 [(1, 0.42), (2, 0.58)]
10 [('science', 0.59362657133

## OK, this works but it needs a speed up: vectorize


In [433]:
test_docs = [
    {'id': 0, 'words': ['A', 'AA']},
    {'id': 1, 'words': ['B', 'BB']},
    {'id': 2, 'words': ['C', 'CC']},
]
words_idx = ['A', 'AA', 'B', 'BB', 'C', 'CC']
nwords = len(words_idx)
ndocs = len(test_docs)
ntopics = 3
beta = 0.01
alpha = 0.5

random_assignment(test_docs, ntopics=ntopics)

W = words_vs_topics_matrix(test_docs, words_idx, ntopics=ntopics)
D = docs_vs_topics_matrix(test_docs, ntopics=ntopics)

print('W=\n', W)
print('D=\n', D)

W=
 [[1. 0. 0.]
 [0. 1. 0.]
 [1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]
 [1. 0. 0.]]
D=
 [[1. 1. 0.]
 [1. 1. 0.]
 [1. 0. 1.]]


In [434]:
# sum W[:, t]  (por columnas)
np.sum(W, axis=0)

array([3., 2., 1.])

In [436]:
# sum D[d, :] (por filas)
np.sum(D, axis=1).reshape((ndocs,1))

array([[2.],
       [2.],
       [2.]])

In [437]:
# (W[w,t] + beta) / (sum(W[:,t]) + beta*nwords) matriz W con cada valor normalizado con la suma de la columna
Wnorm = (W + beta) / (np.sum(W, axis=0) + beta*nwords)
Wnorm

array([[0.33006536, 0.00485437, 0.00943396],
       [0.00326797, 0.49029126, 0.00943396],
       [0.33006536, 0.00485437, 0.00943396],
       [0.00326797, 0.49029126, 0.00943396],
       [0.00326797, 0.00485437, 0.95283019],
       [0.33006536, 0.00485437, 0.00943396]])

In [439]:
# (D[d,t] + alpha) / (sum(D[d,:]) + alpha*ntopics)  matriz D con cada valor normalizadp con la suma de la fila
Dnorm = (D + alpha) / (np.sum(D, axis=1).reshape((ndocs,1)) + alpha*ntopics) 
Dnorm

array([[0.42857143, 0.42857143, 0.14285714],
       [0.42857143, 0.42857143, 0.14285714],
       [0.42857143, 0.14285714, 0.42857143]])

In [440]:
# Ahora hay que "combinar" ambas matrices W-norm y D-norm de una manera especial: 
# ambas matrices tienen el mismo número de columnas, una por topic. Cada columna j de la matriz W-norm se tiene 
# que combinar con la misma columna j de la matriz D-norm (mismo topic). 
# Combinar: por cada elemento de la columna de W-norm multiplicarlo por cada elemento de la columna 
# de D-norm, generando en total nwords*ndocs elementos. Esto se consige "expandiendo" una de las columnas 
# tantas veces como elementos tiene la otra y luego multiplicando. 
# con esto conseguimos una matriz P[nwords, ndocs, ntopics]

In [441]:
P = np.stack([Dnorm]*Wnorm.shape[0], axis=1) * Wnorm
P

array([[[0.14145658, 0.00208044, 0.00134771],
        [0.00140056, 0.21012483, 0.00134771],
        [0.14145658, 0.00208044, 0.00134771],
        [0.00140056, 0.21012483, 0.00134771],
        [0.00140056, 0.00208044, 0.1361186 ],
        [0.14145658, 0.00208044, 0.00134771]],

       [[0.14145658, 0.00208044, 0.00134771],
        [0.00140056, 0.21012483, 0.00134771],
        [0.14145658, 0.00208044, 0.00134771],
        [0.00140056, 0.21012483, 0.00134771],
        [0.00140056, 0.00208044, 0.1361186 ],
        [0.14145658, 0.00208044, 0.00134771]],

       [[0.14145658, 0.00069348, 0.00404313],
        [0.00140056, 0.07004161, 0.00404313],
        [0.14145658, 0.00069348, 0.00404313],
        [0.00140056, 0.07004161, 0.00404313],
        [0.00140056, 0.00069348, 0.4083558 ],
        [0.14145658, 0.00069348, 0.00404313]]])

In [442]:
P[1,5,1]

0.002080443828016643

In [443]:
# primero normalizamos la matriz P para que se convierta en una matriz de probabilidades (filas sumen 1)
Pnorm = P/np.sum(P, axis=2).reshape((ndocs, nwords, 1))
Pnorm

array([[[0.97633876, 0.0143593 , 0.00930194],
        [0.00657932, 0.98708964, 0.00633104],
        [0.97633876, 0.0143593 , 0.00930194],
        [0.00657932, 0.98708964, 0.00633104],
        [0.01003269, 0.01490294, 0.97506437],
        [0.97633876, 0.0143593 , 0.00930194]],

       [[0.97633876, 0.0143593 , 0.00930194],
        [0.00657932, 0.98708964, 0.00633104],
        [0.97633876, 0.0143593 , 0.00930194],
        [0.00657932, 0.98708964, 0.00633104],
        [0.01003269, 0.01490294, 0.97506437],
        [0.97633876, 0.0143593 , 0.00930194]],

       [[0.96760035, 0.00474359, 0.02765605],
        [0.01855408, 0.92788414, 0.05356178],
        [0.96760035, 0.00474359, 0.02765605],
        [0.01855408, 0.92788414, 0.05356178],
        [0.00341226, 0.00168956, 0.99489818],
        [0.96760035, 0.00474359, 0.02765605]]])

## Problema

Pnorm nos permite seleccionar aleatoriamente el nuevo topic para cada palabra de cada documento. El problema que tenemos es que si un documento contiene la misma palabra más de una vez, la representacion "bag of words" que estábamos usando no nos sirve, cada ocurrencia de la misma palabra deberíamos tratarla por separado (pueden venir de distintos topics). Esto complica el trabajar con matrices, ya que la alternativa es representar un documento como una lista de longitud variable de vectores "one hot encoding" (un vector por palabra).

-- Representación "Bag of Words": 
DW = [[2,0,1], # el doc 0 contiene dos veces la palabra 0 y 1 vez la palabra 2
      [0,2,0], # el doc 1 contiene dos veces la palabra 1
      [1,1,0]] # ...

-- Representación one hot encoding de la misma matriz
DWb = [[[1,0,0], [1,0,0], [0,0,1]],  # dos vectores distintos para las dos ocurrencias de la palabra 0 
       [[0,1,0], [0,1,0]],
       [[1,0,0], [1,0,0]]
      ]


Podemos seguir de dos/tres formas:

0. Continuar con la representación "bag of words" y asumir que el algoritmo puede no converger bien.

1. Versión secuencial: iterar para cada documento d, para cada palabra w. Pnorm[d][w] contiene el vector de probabilidades para seleccionar el nuevo topic. O(ndocs * nwordsmedio * O(random.choice(ntopics)) )

2. Versión matricial con sparse matrix: el problema aquí es que cada documento tiene longitud variable de palabras. La solucion bestia: calcular el documento más largo y completar el resto con vectores vacíos. Esto obliga en la práctica a usar sparse matrices.


## Opción 0

Si tengo 5 ocurrencias de la misma palabra en un documento, selecciono el mismo nuevo topic para todas ellas.

Para vectorizar la seleccion aleatoria del nuevo topic (para cada palabra de cada documento) se necesita una función random.choice() en paralelo

https://stackoverflow.com/a/57238866

In [444]:
# generamos la suma cumulativa por filas
Pnormcum = Pnorm.cumsum(axis=2)
Pnormcum

array([[[0.97633876, 0.99069806, 1.        ],
        [0.00657932, 0.99366896, 1.        ],
        [0.97633876, 0.99069806, 1.        ],
        [0.00657932, 0.99366896, 1.        ],
        [0.01003269, 0.02493563, 1.        ],
        [0.97633876, 0.99069806, 1.        ]],

       [[0.97633876, 0.99069806, 1.        ],
        [0.00657932, 0.99366896, 1.        ],
        [0.97633876, 0.99069806, 1.        ],
        [0.00657932, 0.99366896, 1.        ],
        [0.01003269, 0.02493563, 1.        ],
        [0.97633876, 0.99069806, 1.        ]],

       [[0.96760035, 0.97234395, 1.        ],
        [0.01855408, 0.94643822, 1.        ],
        [0.96760035, 0.97234395, 1.        ],
        [0.01855408, 0.94643822, 1.        ],
        [0.00341226, 0.00510182, 1.        ],
        [0.96760035, 0.97234395, 1.        ]]])

In [445]:
# para cada una de las filas generamos un numero aleatorio entre 0 y 1
r = np.random.rand(Pnorm.shape[0], Pnorm.shape[1], 1)
r

array([[[0.1698489 ],
        [0.46895872],
        [0.93194399],
        [0.31099902],
        [0.41632183],
        [0.15577227]],

       [[0.69572806],
        [0.46911913],
        [0.93911042],
        [0.25447041],
        [0.21976453],
        [0.79874256]],

       [[0.74613682],
        [0.21550664],
        [0.60849858],
        [0.37751744],
        [0.65856802],
        [0.41614583]]])

In [446]:
# y ahora el truco que me encanta
q = Pnormcum >= r
q

array([[[ True,  True,  True],
        [False,  True,  True],
        [ True,  True,  True],
        [False,  True,  True],
        [False, False,  True],
        [ True,  True,  True]],

       [[ True,  True,  True],
        [False,  True,  True],
        [ True,  True,  True],
        [False,  True,  True],
        [False, False,  True],
        [ True,  True,  True]],

       [[ True,  True,  True],
        [False,  True,  True],
        [ True,  True,  True],
        [False,  True,  True],
        [False, False,  True],
        [ True,  True,  True]]])

In [448]:
Tnew = np.argmax(Pnormcum >= r, axis=-1).reshape((ndocs, nwords, 1))
Tnew

array([[[0],
        [1],
        [0],
        [1],
        [2],
        [0]],

       [[0],
        [1],
        [0],
        [1],
        [2],
        [0]],

       [[0],
        [1],
        [0],
        [1],
        [2],
        [0]]])

In [449]:
# en realidad sin el reshape queda mejor. Tnew[nwords, ndocs]:
Tnew = np.argmax(Pnormcum >= r, axis=-1)

d = 0  # document 0
w = 0  # word 0
Tnew[d,w]

0

In [450]:
# ahora falta actualizar las matrices W y D
Tnew

array([[0, 1, 0, 1, 2, 0],
       [0, 1, 0, 1, 2, 0],
       [0, 1, 0, 1, 2, 0]])

## Opción 1

Versión secuencial

## Opción 2

Sparse matrices